<a href="https://colab.research.google.com/github/gnuevo/Itroduccion-GANs/blob/master/GaussianGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a los GANs -- En colaboración con VigoBrains

__Gregorio Nuevo Castro__

Este documento está basado en un workshop previo ([código aquí](https://github.com/Machine-Learning-Tokyo/Intro-to-GANs/blob/master/GaussianGAN.ipynb)).

## Introducción

Bienvenido a esta _notebook_. En ella vamos a presentar brevemente el concepto de GANs y a ejecutar nuestro primer GAN. Pero antes de nada vamos a presentar las herramientas que vamos a utilizar:

+ **Colab**, este entorno de ejecución parecido a las Jupyter notebooks ([https://colab.research.google.com/]()).
+ **TensorFlow**, un framework para definir y ejecutar redes neuronales ([https://www.tensorflow.org/]()).

## Importar dependencias

Ahora importamos las dependencias que utilizaremos a lo largo del documento.

In [0]:
# importar dependencias
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, BatchNormalization, UpSampling2D, Conv2D, LeakyReLU, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from google.colab import widgets
import matplotlib.gridspec as gridspec
from matplotlib import animation, rc
from IPython.display import HTML
from PIL import Image

## Ejercicio propuesto

Vamos a llevar a cabo un ejercicio muy simple. Queremos implementar un GAN para aproximar una [función Gaussiana](https://es.wikipedia.org/wiki/Funci%C3%B3n_gaussiana) en una dimensión. En la siguiente imagen podemos ver distintos ejemplos de funciones Gaussianas.
![Funciones Gaussianas](https://camo.githubusercontent.com/4c94d1fc9852cdcbff7021b00f799d2f0aaf3e94/68747470733a2f2f75706c6f61642e77696b696d656469612e6f72672f77696b6970656469612f636f6d6d6f6e732f7468756d622f372f37342f4e6f726d616c5f446973747269627574696f6e5f5044462e7376672f3130383070782d4e6f726d616c5f446973747269627574696f6e5f5044462e7376672e706e67)

Normalmente los GANs reciben un ruido de entrada z que sigue una distribución Uniforme o bien Gaussiana. En este ejercicio vamos a elegir una distribución Uniforme. Por qué? Bueno, si queremos aproximar una función Gaussiana a partir de otra Gaussiana, la transformación puede resultar trivial. En cambio, si introducimos un ruido z uniforme obligamos al GAN a aprender la transformación no uniforme de pasar de la forma de una distribución uniforme a la forma de una distribución Gaussiana.

## Empezando con los GANs

Tras lo que hemos hablado en la charla, ya deberías estar un poco familiarizado con los GANs y con como funcionan. En resumidas cuentas, necesitamos dos redes neuronales que llamamos **Generador** y **Discriminador**. El **Generador** intenta imitar el tipo de muestras que hay en el dataset; mientras que el **Discriminador** intenta adivinar si una muestra en concreto es verdadera (legítima) o es falsa (creada por el **Generador**). 

Por ejemplo, en el caso de que en el dataset haya fotos de perros y gatos. El **Generador** intentaría aprender a generar fotos que parecen perros o gatos; mientras que el **Discriminador** aprendería a distinguir si una imagen concreta es real (pertenece al dataset) o falsa (es producto del **Generador**)

Como sabemos, un GAN está formado por una serie de elementos:

+ Nuestra **distribución objetivo**. Ésta es la distribución de probabilidad que queremos imitar, es la distribución de probabilidad que aparece en el dataset. En el ejemplo de los perros y gatos sería la distribución de probabilidad de los píxeles para hacer que una imagen parezca un perro o un gato. Recuerda p(y|x) y p(x,y). En nuestro ejemplo concreto la distribución de probabilidad que queremos imitar es una función Gaussiana.
+ Un **muestreador de ruido**. Ésta es la entrada a nuestro **Generador**. Necesitamos un generador de números aleatorios que siga una distribución dada. En nuestro caso hemos dicho que vamos a utilizar una distribución Uniforme. Podemos hacer que esta distribución tenga tantas dimensiones como queramos. En general, será un vector de N dimensiones de distribución uniforme. Esto es ALGO IMPORTANTE QUE DEBES TENER EN MENTE.
+ El **Discriminador**, como ya sabemos es una red neuronal que distingue entre muestras reales (vienen del dataset) o falsas (son producto del generador).
+ El **Generador**, es una red neuronal que aprende a producir muestras que parecen reales a partir de ruido no correlacionado.

Al principio puede resultar extraño entender qué son el **Generador** y el **Discriminador**. Es decir, hacen cosas que molan mucho. Entonces, tienen algo de especial? No, para nada! El **Discriminador** es un simple classificador binario: 0 si creo que la muestra es falsa; 1 si creo que es verdadera. Y el **Generador** es una red neuronal que recibe ruido de entrada y devuelve un tensor con la misma forma que las muestras en el dataset. En ese caso, ¿dónde está la mágia de los GANs? El truco está en como interactúan el **Generador** y el **Discriminador** entre sí para llevar a cabo su tarea.

### Distribución objetivo y ruido de entrada

Ahora que ya sabemos bastante sobre nuestro GAN llega el momento de empezar a escribir algo de código. Lo primero que vamos a hacer es definir funciones para generar la distribución objetivo y la distribución de ruido de entrada.

+ **Distribución objetivo**, función Gaussiana de la que podemos elegir la _media_ y la desviación estándar _std_.
+ **Distribución de ruido**, muestras uniformes entre 0 y 1.

In [0]:
### Uniform sampler ###
# Our noise sampler (Z) is a function that returns a vector of a desired length 
# with uniform samples between 0 and 1
# we can use m and n to ask for a matrix of Uniform samples with the desired shape
# therefore m is the dimension of the noise
def uniform_sampler():
  return lambda m, n: np.random.uniform(low=0.0, high=1.0, size=(m, n))

# we set the sampler to a variable so we can change it later if we want
noise_sampler = uniform_sampler()
# this is a 10x5 matrix with Uniform samples
uniform_matrix = noise_sampler(10, 5)


### Gaussian sampler ###
# Our Gaussian sampler is a function that returns a vector of Gaussian samples
# with a specific mean and std
# We can use n to ask for a vector of shape 1xn
def gaussian_sampler(mu, sigma):
  return lambda n: np.random.normal(mu, sigma, (1, n))

# again, set it to a variable so we can change it later
mu, sigma = 2, 1
data_sampler = gaussian_sampler(mu, sigma)
# this is a 1x5 vector with Gaussian samples
gaussian_vector = data_sampler(5)

Ahora podemos echarle un vistazo a los datos generados utilizando histogramas. Nuestras funciones devuelven matrices de números, pero a través de los histogramas deberíamos ser capaces de observar las formas características de las distribuciones Gaussiana y Uniforme.

A través del siguiente formulario de colab puedes configurar algunos valores de la distribución para ajustarla a gusto.

+ **num_samples**, número de muestras a dibujar. Cuántas más muestras se dibujen más se pareceran los histogramas a la distribución de probabilidad correcta.
+ **gaussian_mean** y **gaussian_std**, parámetros de la distribución Gaussiana
+ **num_bins**, número de bloques del histograma
+ **colores**, echa un vistazo y elije tus favoritos ;)

Experimenta cambiando los valores de los parámetros y viendo cómo afectan al resultado. Es importante que te familiarices con estos conceptos de cara a los siguientes pasos.

In [0]:
#@title Define parameters
num_samples = 6761 #@param {type:"slider", min:1, max:10000, step:10}
gaussian_mean = 3 #@param {type:"slider", min:-3, max:3, step:0.2}
gaussian_std = 2.7 #@param {type:"slider", min:0.1, max:3, step:0.1}

num_bins = 100 #@param {type:"slider", min: 1, max:100, step:1}
my_favourite_color_is = 'red' #@param ["green", "blue", "red", "yellow", "black", "cyan", "magenta"] {allow-input: false}
but_I_also_like = 'blue' #@param ["green", "blue", "red", "yellow", "black", "cyan", "magenta"] {allow-input: false}

# this creates a grid to make 2 plots side by side
grid = widgets.Grid(1,2)

# generate samples with the chosen characteristics
noise = noise_sampler(1, num_samples)
noise = np.reshape(noise, num_samples)
data_sampler = gaussian_sampler(gaussian_mean, gaussian_std)
data = data_sampler(num_samples)
data = np.reshape(data, num_samples)

# plot our data
with grid.output_to(0, 0):
  n, bins, patches = plt.hist(noise, num_bins, density=True, facecolor=my_favourite_color_is,
                                alpha=0.75)
  ax = plt.gca()
  ax.set_xlim(-0.25,1.25)
  ax.set_ylim(0, 1.5)
  plt.title("Noise distribution (Uniform between 0 and 1)")
  
with grid.output_to(0, 1):
  n, bins, patches = plt.hist(data, num_bins, density=True, facecolor=but_I_also_like,
                                alpha=0.75)
  ax = plt.gca()
  ax.set_xlim(-10,10)
  ax.set_ylim(0, 0.7)
  plt.title("Data distribution (Gaussian with mean={} and std={})".format(gaussian_mean, gaussian_std))


Espero que ahora tengas mucho más claro los tipos de distribuciones con los que estamos trabajando.

## Definir las redes neuronales

Vamos a programar el **Discriminador** y el **Generador**.

### Discriminador

El **Discriminador** tiene que distinguir si una muestra es real (1) o falsa (0). ¿Suena familiar? Eso es un clasificador binario. De hecho, nuestro **Discriminador** no es más que un clasificador binario.

Aquí tenemos que hacer una pequeña pausa para ver el mundo desde el punto de vista del **Discriminador**. ¿Qué sucederá si el **Discriminador** recibe muestras (tanto reales como falsas) una a una? Pues que difícilmente será capaz de hacerse una idea de la distribución que siguen. Podemos verlo nosotros mismos con la herramienta de visualización de arriba. Reduce el valor de `num_bins` paulatinamente hasta llegar a `1`. Cuanto más pequeño es el valor de `num_bins` más difícil es distinguir la forma verdadera de la distribución. Ahora reduce también `num_samples` a `1`. Con esta configuración es completamente imposible distinguir ambas distribuciones. Puede que un humano sea capaz de hacerlo, pero eso es probablemente porque nosotros contamos con memoria. El **Discriminador** carece de ningún tipo de memoria, por lo que bajo estas condiciones le resultará muy difícil distinguir entre muestras reales y muestras falsas.

Si el **Discriminador** recibe las muestras una a una, probablemente no haga un buen trabajo. Para ello podemos darle las muestras en grupos. Si el **Discriminador** ve un grupo de muestras de una Gaussiana será capaz de observar la forma característica de la distribución. Al igual que a nosotros nos resulta mucho más informativo un histograma de muchas muestras que no observar muestras una a una. A esta técnica se la llama _minibatch discrimination_ y la vamos a utilizar para mejorar el funcionamiento de nuestro **Discriminador**.

Ahora podemos programar nuestro **Discriminador** con las siguientes características:

+ Acepta una entrada con forma `minibatch_size` (el grupo de muestras de nuestro _minibatch discrimination_)
+ Tiene 3 capas _fully connected_
+ Las activaciones son `elu` para las capas intermedias y `sigmoid` para la capa de salida



In [0]:
def create_discriminator(input_size=100, hidden_size=50, output_size=1):
  return Sequential([
    Dense(hidden_size, activation='elu', input_shape=(input_size,)),
    Dense(hidden_size, activation='elu'),
    Dense(1, activation='sigmoid')
  ])

### Generador

En cuanto al **Generador** tenemos lo siguiente. Se trata de una red que recibe un vector de ruido Uniforme (de N dimensiones) y devuelve lo que sería una muestra falsa. Vamos a definir nuestro **Generador** con una arquitectura similar al **Discriminador**. En concreto, nuestro **Generador**

+ Recibe 
+ Tiene 3 capas _fully connected_.
+ La activación es `elu` para las capas internas, y `linear` para la última capa. ¿Por qué la última capa no tiene activación? Bueno, las funciones de activación normalmente limitan el rango de valores de la salida (por ejemplo `relu` y `elu` a valores positivos, `sigmoid` a valores entre 0 y 1, etc.). Eso no es lo que nos interesa. En este caso, si queremos imitar una función uniforme sabemos que el rango de valores potencial es desde `-infinito` a `+infinito++.

In [0]:
def create_generator(input_size=1, hidden_size=50, output_size=100):
  return Sequential([
    Dense(hidden_size, activation='elu', input_shape=(input_size,)),
    Dense(hidden_size, activation='elu'),
    Dense(output_size, activation='linear')
  ])

### Compilar los modelos

En `TensorFlow` y otros _frameworks_ para ejecutar redes neuronales, tenemos que compilar los modelos antes de utilizarlos. Es decir, para utilizar nuestro **Generador** y nuestro **Discriminador** no sólo vale con definir su arquitectura, si no que también necesitamos compilarlo. Al compilarlo, se crea el grafo de ejecución de la red neuronal y se realizan optimizaciones para acelerar su ejecución, entre otras. (Nota, no todos los _frameworks_ necesitan de este paso de compilado; algunos, como `PyTorch` carecen de él; el mismo `TensorFlow` está experimentando con un modo de ejecución llamado `Eager Execution` que también carece de este paso de compilación).

Compilar las partes de un GAN resulta un proceso un tanto verboso. No sólo hay que compilar **Discriminador** y **Generador** por su lado, si no también un modelo conjunto o combinado (`combined`). Este modelo combinado será el encargado de entrenar el **Generador**. ¿Por qué? Pues porque para entrenar el **Generador** necesitamos _combinar_ tanto el **Generador** en sí (para producir una muestra) como el **Discriminador** (para que haga una valoración de la muestra producida).

In [0]:
def compile_models(generator, discriminator, noise_length):
  assert generator.layers[-1].output.shape[1] == discriminator.layers[0].input.shape[1], ("The output shape of generator must match the input shape of discriminator")
  
  adam = Adam(lr=0.0001, beta_1=0.5)

  generator.compile(loss='binary_crossentropy', optimizer=adam)

  discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

  z = Input(shape=(noise_length,))
  img = generator(z)
  discriminator.trainable = False
  score = discriminator(img)

  combined = Model(inputs=[z], outputs=[score])
  combined.compile(loss='binary_crossentropy', optimizer=adam)

  return generator, discriminator, combined  

## Entrenamiento



### Notas sobre minibatch discrimination y la implementación en Keras

Hemos hablado arriba sobre `minibatch discrimination`. Este proceso nos permite hacer la discriminación de muestras sobre todo un minibatch en lugar de hacerlo muestra a muestra. Así, debería resultar mucho más fácil distinguir las distribuciones de probabilidad real y falsa, y por tanto esto ayuda al entrenamiento.

Esto significa que nuestro **Discriminador**, en lugar de recibir una única muestra y devolver un score (1 -> 1) recibiría, por ejemplo, un minibatch de 100 muestras y devolvería un score (100 -> 1). Esto, a priori, no cambia el comportamiento de nuestro **Generador**. El **Generador** recibe una muestra de ruido Uniforme (de N dimensiones) y genera una **única** muestra de la distribución falsa a partir de ella (N -> 1).

Esto funciona muy bien mientras estamos entrenando el **Discriminador**. Entonces podemos generar `batch_size` muestras utilizando el **Generador** y dárselas al **Discriminador** para que las valore. Lamentablemente, esto no funciona cuando para entrenar el **Generador** dado que tenemos que utilizar el modelo **Combinado**. Imagínatelo, para entrenar el **Generador** a través del modelo combinado en el caso anterior produciríamos 100 muestras Uniformes (de N dimensiones), con ellas el **Generador** produciría 100 muestras falsas y el **Discriminador** cogería esas 100 muestras de golpe para darnos una valoración.  Entonces nuestro modelo **Combinado** sería tal que

```
              +------------+                 +---------------+
              |            |                 |               |
  Uniforme +--> Generador  +--> Fake dist +--> Discriminador +-->  Score
  (100, N)    |            |    (100,)       |               |     (1,)
              +------------+                 +---------------+

```

La entrada equivale a un minibatch de 100 muestras (100, N) mientras que la salida equivale a un minibatch de 1 muestra (1,). Esto no está permitido en Keras (aunque sí lo está en PyTorch, puedes echar un vistazo al [código en el que se basa este ejercicio](https://github.com/Machine-Learning-Tokyo/Intro-to-GANs/blob/master/GaussianGAN.ipynb)). Por tanto se ha decidido cambiar el código para que el **Generador**, en lugar de generar las muestras una a una (1 -> 1) lo haga minibatch a minibatch a partir de una única muestra uniforme (1 -> 100). 

```
              +------------+                 +---------------+
              |            |                 |               |
  Uniforme +--> Generador  +--> Fake dist +--> Discriminador +-->  Score
  (1, N)      |            |    (100,)       |               |     (1,)
              +------------+                 +---------------+

```

Este cambio nos permite escribir el código en Keras. Sin embargo, al producir minibatch muestras a la vez con el **Generador**, todas estas muestras estarán correlacionadas. Esto es algo que en principio no es deseable. Por otra parte. Al producir las muestras minibatch a minibatch, al **Generador** le resulta mucho más fácil entender las características de la distribución que debe imitar y por tanto facilita el entrenamiento.

### Utility functions

Estas funciones nos permiten realizar _plotting_ de los resultados. No hace falta que las entiendas pero por si estás interesado

+ `make_histogram()` coge dos arrays con datos sobre una y otra distribución (real y falsa) y dibuja el histograma de ambas. En este caso, el histograma nos ayuda a observar la distribución de probabilidad de los datos.

+ `plot_errors()` dibuja las gráficas de error de los errores del discriminador y del generador.


In [0]:
# utility function to plot the histograms of training
def make_histogram(data_real, data_fake, i, n_bins=100):
    fig = plt.figure(figsize=(8, 8))
    axes = plt.subplot(111)
    n, bins, patches = plt.hist(data_real, n_bins, normed=1, facecolor='green',
                                alpha=0.75, label="Real data")
    n, bins, patches = plt.hist(data_fake, bins, normed=1, facecolor='blue',
                                alpha=0.75, label="Generated data")
    axes.set_xlim(-5, 8)
    axes.set_ylim(0, 1)
    plt.title("Probability distribution of real and fake data")
    plt.legend(["Real data", "Generated data"])
    plt.tight_layout()

# utility function to plot the evolution of the errors during training
def plot_errors(real_loss_buffer, fake_loss_buffer, g_loss_buffer):
    fig = plt.figure(0, figsize=(10, 10))
    plt.subplot(311)
    plt.plot(real_loss_buffer)
    plt.title("D real loss")

    plt.subplot(312)
    plt.plot(fake_loss_buffer)
    plt.title("D fake loss")

    plt.subplot(313)
    plt.plot(g_loss_buffer)
    plt.title("G fake loss")

### Hiperparámetros

El siguiente formulario nos permite cambiar algunos de los hiperparámetros de los que depende el sistema.

La configuración por defecto funciona correctamente, pero puedes cambiarlos para ver como influencian el entrenamiento.

In [0]:
from collections import namedtuple
param = namedtuple("Parameters",
                  "iterations mu sigma hidden_size batch_size input_size")

#@title Define hyperparameters
param.iterations = 19000 #@param {type:"slider", min:1000, max:50000, step:2000}
param.print_step = 400 #@param {type:"slider", min:50, max:1000, step:50}
param.mu = 1.5 #@param {type:"slider", min:-5, max:5, step:0.1}
param.sigma = 1 #@param {type:"slider", min:0.1, max:3, step:0.1}
param.hidden_size = 50 #@param {type:"slider", min:10, max:100, step:5}
param.batch_size = 100 #@param {type:"slider", min:10, max:1000, step:10}
param.noise_size = 1 #@param {type:"slider", min:1, max:10, step:1}


# print some results every 200 steps
print_step = 200

# these are some hyperparameters to for the network architecture
hidden_size = 50
output_size = 1

learning_rate = 2e-4
batch_size = 100
test_num = 10000 # number of samples used for testing

### Función de entrenamiento

In [0]:
def train():
  # this variable is for display purposes, you can ignore it!
  grid = widgets.Grid(2,2)
  global train_gaussian, train_generation, train_errors
  train_generation = []
  train_errors = []

  # create and compile networks
  discriminator = create_discriminator(param.batch_size, param.hidden_size, 1)
  generator = create_generator(param.noise_size, param.hidden_size, param.batch_size)
  G, D, C = compile_models(generator, discriminator, param.noise_size)
  print('G', G.input_shape, '->', G.output_shape)
  print('D', D.input_shape, '->', D.output_shape)
  
  # get our data and noise samplers
  data_sampler = gaussian_sampler(param.mu, param.sigma)
  test_data = data_sampler(test_num)
  train_gaussian = test_data
    
  # this list will keep track of the errors during training
  real_loss_buffer = []
  fake_loss_buffer = []
  g_loss_buffer = []
  
  global count
  count = 0
  
  for i in range(param.iterations):
    ## Train Discriminator on real samples
    # take real samples
    real_data = data_sampler(param.batch_size)
    # train discriminator on real images
    d_real_targets = np.ones(1)
    real_loss = D.train_on_batch(real_data, d_real_targets)
    real_loss_buffer.append(real_loss[0])
    
    ## Train Discriminator on fake samples
    z = noise_sampler(1, param.noise_size)
    fake_data = G.predict(z)
    d_fake_targets = np.zeros(len(fake_data))
    fake_loss = D.train_on_batch(fake_data, d_fake_targets)
    fake_loss_buffer.append(fake_loss[0])
    
    ## Train Generator
    z = noise_sampler(1, param.noise_size)
    g_targets = np.ones(1)
    g_loss = C.train_on_batch(z, g_targets)
    g_loss_buffer.append(g_loss)
    
    
    ## Visualise training--------------
    if (i + 1) % param.print_step == 0:

      # print current errors
      with grid.output_to(1,0):
        if (i + 1) % (15 * param.print_step) == 0:
          grid.clear_cell()
        print("{:>6} -> D_real_loss {:.6f}  D_fake_loss {:.6f}  "
              "G_fake_loss {:.6f}".format(
          i+1,
          real_loss_buffer[-1],
          fake_loss_buffer[-1],
          g_loss_buffer[-1]
        ))
          
      # plot distributions
      fake_data = np.array([[]]) # empty array
      for _ in range(test_num // param.batch_size):
        z_test = noise_sampler(1, param.noise_size)
        new_fake_data = G.predict(z_test)
        fake_data = np.concatenate((fake_data, new_fake_data), axis=1)
      
      with grid.output_to(0, 0):
        grid.clear_cell()
        
        make_histogram(
            test_data[0,:], 
            fake_data[0,:], 
            i)
        plt.savefig("figure_{}.png".format(count))
        count += 1

      # plot training errors
      with grid.output_to(0, 1):
        grid.clear_cell()
        plot_errors(real_loss_buffer, fake_loss_buffer, g_loss_buffer) 

### Ejecutar el código

Ahora que ya está todo preparado, podemos ejecutar el código.

In [0]:
train()

### Análisis de los resultados

Según el GAN empiece a entrenar verás 3 cuadrados con información:

+ Arriba a la izquierda se muestra el histograma de ambas distribuciones (o lo que es lo mismo, la distribución de probabilidad de los datos reales y falsos) en el instante concreto de entreanmiento. Si todo va bien, la gráfica azul debería parecerse cada vez más a la verde.
+ Arriba a la derecha se muestran las gráficas de errores y su evolución en el tiempo: el error del discriminador al clasificar muestras reales, el error del discriminador al clasificar muestras falsas, y el error del generador.
+ Abajo a la izquierda se muestra el valor numérico de los errores según la iteración actual.

#### Cosas en las que fijarse

+ ¿Cómo aprende el GAN?
+ ¿En cada nueva iteración, el GAN lo hace siempre mejor que en las versiones anteriores o hay veces que no?
+ ¿Qué pasa con los _plots_ de los errores? ¿Cómo es que a veces suben, otras veces bajan, a veces parece que oscilan como locos?
+ Si los errores son tan extraños ¿cómo sé cuándo puedo parar de entrenar el GAN? ¿cuándo termino el entrenamiento?
+ ¿Si entrenamos el GAN hasta la perfección (las distribuciones real y falsa completamente indistinguibles) cuál será el error de l **Discriminador**?

### Volver a probar

Para que no tengas que ejecutar varias celdas para entrenar, aquí se ha puesto todo en una única celda. Simplemente configura los hiperparámetros que quieras y ejecuta la celda para entrenar el GAN.

In [0]:
from collections import namedtuple
param = namedtuple("Parameters",
                  "iterations mu sigma hidden_size batch_size input_size")

#@title Define hyperparameters
param.iterations = 19000 #@param {type:"slider", min:1000, max:50000, step:2000}
param.print_step = 400 #@param {type:"slider", min:50, max:1000, step:50}
param.mu = 3.2 #@param {type:"slider", min:-5, max:5, step:0.1}
param.sigma = 0.4 #@param {type:"slider", min:0.1, max:3, step:0.1}
param.hidden_size = 50 #@param {type:"slider", min:10, max:100, step:5}
param.batch_size = 100 #@param {type:"slider", min:10, max:1000, step:10}
param.noise_size = 1 #@param {type:"slider", min:1, max:10, step:1}


# print some results every 200 steps
print_step = 200

# these are some hyperparameters to for the network architecture
hidden_size = 50
output_size = 1

learning_rate = 2e-4
batch_size = 100
test_num = 10000 # number of samples used for testing

train()

## Hacer un vídeo

Ahora vamos a visualizar el entrenamiento haciendo una animación. Mientras se estaba entrenando el GAN, hemos hecho capturas de las imágenes. Veamos cuantas tenemos guardadas.

In [0]:
!ls
!ls | wc -l
print("Number of images saved", count)

### Reseteo de emergencia

El siguiente formulario es un reseteo de emergencia. Es posible que haya habido algún fallo y hayas tenido que reiniciar el entorno de ejecución. Si ha sido así, se ha perdido información de contexto relevante para generar la animación. Marcando el `restart_checkbox` y ejecutando el siguiente código se debería recuperar esa información de contexto automáticamente.

In [0]:
#@title Did you have to restart? (check and run if you did)
restart_checkbox = False #@param {type:"boolean"}

if restart_checkbox:
  # import again
  print("Importing modules")
  import numpy as np
  import matplotlib.pyplot as plt
  from google.colab import widgets
  import matplotlib.gridspec as gridspec
  from matplotlib import animation, rc
  from IPython.display import HTML
  from PIL import Image
  
  # read files
  print("Reading the number of images")
  from os import listdir
  count = len(list(filter(lambda name: "figure_" in name, listdir())))
  print("Number of images saved", count)

### El objeto de animación

La siguiente celda contiene el objeto encargado de realizar la animación. Tan solo tienes que ejecutarla.

In [0]:
class AnimObject(object):
    def __init__(self, images):
        print(len(images))
        self.fig, self.ax = plt.subplots()
        self.ax.set_title("")
        self.fig.set_size_inches((8, 8))
        self.plot = plt.imshow(images[0])
        plt.tight_layout()
        self.images = images
        
    def init(self):
        self.plot.set_data(self.images[0])
        self.ax.grid(False)
        return (self.plot,)
      
    def animate(self, i):
        self.plot.set_data(self.images[i])
        self.ax.grid(False)
        self.ax.set_xticks([])
        self.ax.set_yticks([])
        self.ax.set_title("index {}".format(i))
        return (self.plot,)

def get_figures(template, indices):
    import os.path
    images = []
    for index in indices:
        if os.path.isfile(template.format(index)):
            images.append(Image.open(template.format(index)))
    return images
  
images = get_figures("figure_{}.png", range(count))
animobject = AnimObject(images)
anim = animation.FuncAnimation(
              animobject.fig,
              animobject.animate,
              frames=len(animobject.images),
              interval=150,
              blit=True)

### Visualizar la animación

Ejecuta la siguiente celda para ver la animación.

In [0]:
HTML(anim.to_jshtml())